In [71]:
import requests
import json
import pandas as pd
from datetime import date
import openpyxl
from clickhouse_connect import get_client
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os
import time

load_dotenv()

True

In [72]:
KeyGuten = os.getenv('KeyGuten')
KeyGiper = os.getenv('KeyGiper')
KeyKitchen = os.getenv('KeyKitchen')

In [78]:


# API endpoint
url = 'https://advert-api.wildberries.ru/adv/v1/promotion/count'

# Headers including the API key for authentication
headers_guten = {
    'Authorization': KeyGuten,
    'Accept': 'application/json'
}

headers_giper = {
    'Authorization': KeyGiper,
    'Accept': 'application/json'
}

headers_kitchen = {
    'Authorization': KeyKitchen,
    'Accept': 'application/json'
}

# Make the GET request
response_guten = requests.get(url, headers=headers_guten)
response_giper = requests.get(url, headers=headers_giper)
response_kitchen = requests.get(url, headers=headers_kitchen)

# Check if the request was successful
if response_guten.status_code == 200 and response_giper.status_code == 200 and response_kitchen.status_code == 200:
    # Parse the JSON response
    data_guten = response_guten.json()
    data_giper = response_giper.json()
    data_kitchen = response_kitchen.json()
    print("Data retrieved successfully")
else:
    print(f"Failed to retrieve data. Status code: {response_guten.status_code,response_giper.status_code,response_giper.status_code }")
    print(f"Response: {response_guten.text, response_giper.text,response_giper.text }")

Data retrieved successfully


In [79]:
# Flatten the JSON data
df_guten = pd.json_normalize(
    data_guten['adverts'],
    record_path='advert_list', 
    meta=['type', 'status', 'count']
)
df_giper = pd.json_normalize(
    data_giper['adverts'],
    record_path='advert_list', 
    meta=['type', 'status', 'count']
)
df_kitchen = pd.json_normalize(
    data_kitchen['adverts'],
    record_path='advert_list', 
    meta=['type', 'status', 'count']
)


# Convert 'changeTime' to datetime format
df_guten['changeTime'] = pd.to_datetime(df_guten['changeTime'])
df_giper['changeTime'] = pd.to_datetime(df_giper['changeTime'])
df_kitchen['changeTime'] = pd.to_datetime(df_kitchen['changeTime'])

# Filter rows where the year of 'changeTime' is 2024 or later
df_guten = df_guten[df_guten['changeTime'].dt.year >= 2024]
df_giper = df_giper[df_giper['changeTime'].dt.year >= 2024]
df_kitchen = df_kitchen[df_kitchen['changeTime'].dt.year >= 2024]

# Reset the index and drop the old index
df_guten = df_guten.reset_index(drop=True)
df_giper = df_giper.reset_index(drop=True)
df_kitchen = df_kitchen.reset_index(drop=True)

# Display the updated DataFrame
#df_guten
#df_giper
#df_kitchen

In [263]:
filename = "Promotion.xlsx"
df.to_excel(filename, index=False)

ValueError: Excel does not support datetimes with timezones. Please ensure that datetimes are timezone unaware before writing to Excel.

In [7]:
unique_change_times = df['changeTime'].unique()
print(unique_change_times)

['2025-02-25T09:56:44.373405+03:00' '2025-02-25T09:56:36.977327+03:00'
 '2025-02-10T15:50:50.654674+03:00' '2025-02-10T15:50:46.711076+03:00'
 '2024-12-28T14:56:51.438476+03:00' '2025-02-10T15:50:45.190525+03:00'
 '2025-02-06T14:45:40.972398+03:00' '2024-12-28T14:59:03.412003+03:00'
 '2025-02-10T15:50:43.275656+03:00' '2025-02-25T10:07:05.519407+03:00'
 '2025-01-06T19:56:55.030411+03:00' '2025-01-31T16:14:36.03518+03:00'
 '2025-02-26T15:24:26.898332+03:00' '2025-01-24T17:28:07.565413+03:00'
 '2025-01-24T17:27:45.032317+03:00' '2025-03-03T10:51:20.471041+03:00'
 '2023-06-09T17:13:45.296789+03:00' '2023-07-17T17:56:44.162491+03:00'
 '2023-09-11T19:47:52.142401+03:00' '2023-09-13T19:02:34.535107+03:00'
 '2023-09-11T17:14:54.50553+03:00' '2023-09-13T14:53:09.428335+03:00'
 '2023-12-11T23:45:14.286592+03:00' '2023-12-11T22:31:06.941346+03:00'
 '2024-07-15T12:16:26.499587+03:00' '2024-04-11T10:53:48.400039+03:00'
 '2024-05-13T10:13:17.846636+03:00' '2024-05-20T16:18:30.678177+03:00'
 '2024-0

In [80]:
# Create chunks of 50 campaign IDs each
chunk_size = 50
campaign_chunks_guten = [df_guten['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df_guten), chunk_size)]
campaign_chunks_giper = [df_giper['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df_giper), chunk_size)]
campaign_chunks_kitchen = [df_kitchen['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df_kitchen), chunk_size)]

# Define the API endpoint
url = "https://advert-api.wildberries.ru/adv/v1/promotion/adverts"

# Define the query parameters
query_params = {
    "order": "create",  # Order by the "change" field
    "direction":"desc"
}


# List to store all campaign data
all_campaign_data_guten = []
all_campaign_data_giper = []
all_campaign_data_kitchen = []


# Iterate over each chunk and send a POST request
for idx, (chunk_guten, chunk_giper, chunk_kitchen) in enumerate(zip(campaign_chunks_guten, campaign_chunks_giper, campaign_chunks_kitchen)):
    
    # Send the POST request
    response_guten = requests.post(url, params=query_params, json=chunk_guten, headers=headers_guten)
    time.sleep(1)
    response_giper = requests.post(url, params=query_params, json=chunk_giper, headers=headers_giper)
    time.sleep(1)
    response_kitchen = requests.post(url, params=query_params, json=chunk_kitchen, headers=headers_kitchen)
    # Add a delay to avoid hitting API rate limits
    time.sleep(1)
    
    # Check the response status
    if response_guten.status_code == 200 and response_giper.status_code == 200 and response_kitchen.status_code == 200:
        # Parse the JSON response
        data_guten = response_guten.json()
        data_giper = response_giper.json()
        data_kitchen = response_kitchen.json()
        all_campaign_data_guten.extend(data_guten)
        all_campaign_data_giper.extend(data_giper)
        all_campaign_data_kitchen.extend(data_kitchen)
        print("Data retrieved successfully")
        #print(f"Response for Chunk {idx + 1}: {data_guten}")
    else:
        print(f"Error for Chunk {idx + 1}: {response_guten.status_code,response_giper.status_code,response_kitchen.status_code}, {response_guten.text,response_giper.text,response_kitchen.text}")
        
        
campaigns_guten = pd.DataFrame(all_campaign_data_guten)
campaigns_giper = pd.DataFrame(all_campaign_data_giper)
campaigns_kitchen = pd.DataFrame(all_campaign_data_kitchen)

campaigns_guten['Project'] = 'WB-GutenTech'
campaigns_giper['Project'] = 'WB-ГиперМаркет'
campaigns_kitchen['Project'] = 'WB-KitchenAid'

# Concatenate the DataFrames
combined_campaigns = pd.concat([campaigns_guten, campaigns_giper, campaigns_kitchen], ignore_index=True)

combined_campaigns['Marketplace'] = 'Wildberries'

print("Combined Campaign Data")
#campaigns_kitchen

Data retrieved successfully
Combined Campaign Data


In [81]:
# Keep only the desired columns
columns_to_keep = ["endTime", "createTime", "startTime","name", "advertId", "status", "type","Project","Marketplace"]
# Ensure filtered_df is a copy of the slice, not a view
filtered_df = combined_campaigns[columns_to_keep].copy()
filtered_df['endTime'] = pd.to_datetime(filtered_df['endTime'], format='mixed').dt.date
filtered_df['createTime'] = pd.to_datetime(filtered_df['createTime'], format='mixed').dt.date
filtered_df['startTime'] = pd.to_datetime(filtered_df['startTime'], format='mixed').dt.date

# Mapping dictionaries for 'status' and 'type'
status_mapping = {
    -1: "Кампания в процессе удаления",
    4: "Готова к запуску",
    7: "Кампания завершена",
    8: "Отказался",
    9: "Идут показы",
    11: "Кампания на паузе"
}

type_mapping = {
    4: "Кампания в каталоге (устаревший тип)",
    5: "Кампания в карточке товара (устаревший тип)",
    6: "Кампания в поиске (устаревший тип)",
    7: "Кампания в рекомендациях на главной странице (устаревший тип)",
    8: "Автоматическая кампания",
    9: "Аукцион"
}

# Replace numeric values with their string descriptions
filtered_df['status'] = filtered_df['status'].replace(status_mapping)
filtered_df['type'] = filtered_df['type'].replace(type_mapping)

# Display the updated DataFrame
filtered_df

,endTime,createTime,startTime,name,advertId,status,type,Project,Marketplace
0,2100-01-01,2025-02-19,2025-02-19,Mr Proper_19.02.2025,23408417,Идут показы,Автоматическая кампания,WB-GutenTech,Wildberries
1,2100-01-01,2025-02-12,2025-03-06,Always_12.02.2025,23230882,Идут показы,Автоматическая кампания,WB-GutenTech,Wildberries
2,2100-01-01,2025-01-27,2025-01-27,AND_Тонометры на запястье_27.01.2025,22846531,Идут показы,Автоматическая кампания,WB-GutenTech,Wildberries
3,2100-01-01,2025-01-27,2025-01-27,AND_Тонометры авто_27.01.2025,22846516,Идут показы,Автоматическая кампания,WB-GutenTech,Wildberries
4,2100-01-01,2025-01-27,2025-01-27,AND_Тонометры мех_27.01.2025,22846446,Идут показы,Автоматическая кампания,WB-GutenTech,Wildberries
...,...,...,...,...,...,...,...,...,...
86,2024-07-23,2024-06-05,2024-06-05,Braun Триммеры,17748698,Кампания завершена,Автоматическая кампания,WB-ГиперМаркет,Wildberries
87,2024-07-23,2024-06-05,2024-06-05,Braun Эпиляторы,17741906,Кампания завершена,Автоматическая кампания,WB-ГиперМаркет,Wildberries
88,2100-01-01,2024-10-30,2025-01-09,Чайники_30.10.2024,21008536,Идут показы,Автоматическая кампания,WB-KitchenAid,Wildberries
89,2100-01-01,2024-10-30,2025-01-09,Миксер планет_30.10.2024,21008249,Идут показы,Автоматическая кампания,WB-KitchenAid,Wildberries


In [305]:
unique_change_times = filtered_df['advertId'].nunique()
print(unique_change_times)

140


In [293]:
filename = "Companias_info.xlsx"
campaigns_df.to_excel(filename, index=False)

In [241]:
# Flatten the JSON response into a DataFrame
df = pd.json_normalize(data, sep='_')

# Display the DataFrame
df

,views,clicks,ctr,cpc,sum,atbs,orders,cr,shks,sum_price,days,boosterStats,advertId,interval_begin,interval_end
0,30450,1762,5.79,3.23,5688.32,302,60,3.41,60,145067,"[{'date': '2025-02-17T03:00:00+03:00', 'views'...","[{'date': '2025-02-17T00:00:00Z', 'nm': 234921...",22846531,2025-02-17,2025-02-19
1,20014,507,2.53,9.35,4742.67,15,0,0.00,0,0,"[{'date': '2025-02-17T03:00:00+03:00', 'views'...","[{'date': '2025-02-17T00:00:00Z', 'nm': 286529...",21577733,2025-02-17,2025-02-19
2,1305,45,3.45,14.07,633.29,8,1,2.22,1,3537,"[{'date': '2025-02-19T03:00:00+03:00', 'views'...","[{'date': '2025-02-19T00:00:00Z', 'nm': 171562...",21554091,2025-02-17,2025-02-19
3,2106,73,3.47,4.05,295.87,3,1,1.37,1,7845,"[{'date': '2025-02-17T03:00:00+03:00', 'views'...","[{'date': '2025-02-17T00:00:00Z', 'nm': 203510...",18447862,2025-02-17,2025-02-19
4,5214,258,4.95,3.70,955.34,9,0,0.00,0,0,"[{'date': '2025-02-17T03:00:00+03:00', 'views'...","[{'date': '2025-02-17T00:00:00Z', 'nm': 161420...",20487381,2025-02-17,2025-02-19
5,15492,669,4.32,9.53,6375.65,58,10,1.49,10,34712,"[{'date': '2025-02-17T03:00:00+03:00', 'views'...","[{'date': '2025-02-17T00:00:00Z', 'nm': 159488...",21965489,2025-02-17,2025-02-19
6,20519,932,4.54,1.51,1404.77,82,4,0.43,4,24196,"[{'date': '2025-02-17T03:00:00+03:00', 'views'...","[{'date': '2025-02-17T00:00:00Z', 'nm': 252275...",19297608,2025-02-17,2025-02-19
7,4032,111,2.75,6.36,705.93,5,1,0.90,1,31018,"[{'date': '2025-02-17T03:00:00+03:00', 'views'...","[{'date': '2025-02-17T00:00:00Z', 'nm': 208813...",20124734,2025-02-17,2025-02-19
8,8157,198,2.43,11.55,2285.94,16,3,1.52,3,23640,"[{'date': '2025-02-17T03:00:00+03:00', 'views'...","[{'date': '2025-02-17T00:00:00Z', 'nm': 266534...",21821064,2025-02-17,2025-02-19
9,12202,826,6.77,3.26,2694.57,48,8,0.97,8,42838,"[{'date': '2025-02-17T03:00:00+03:00', 'views'...",NaN,19276694,2025-02-17,2025-02-19


In [172]:
import requests

# Define the API endpoint
url = "https://advert-api.wildberries.ru/adv/v1/promotion/adverts"

# Define the query parameters
query_params = {
    "order": "change",  # Order by the "change" field

}

# Define the request body (list of campaign IDs)
# Replace with actual campaign IDs (maximum 50 IDs)
request_body = [20955474]

# Define the headers (replace 'your_api_token' with your actual API token)
headers = {
    "Authorization": KeyGuten,  # Replace with your API token
    "Content-Type": "application/json"
}

# Make the POST request
response = requests.post(url, params=query_params, json=request_body, headers=headers)

# Check the response status
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()
    print("Response Data:", data)
else:
    print(f"Error: {response.status_code}, {response.text}")

Response Data: [{'endTime': '2024-10-31T19:09:55.666795+03:00', 'createTime': '2024-10-28T18:49:38.583434+03:00', 'changeTime': '2024-10-31T19:09:55.666795+03:00', 'startTime': '2024-10-31T12:15:15.669999+03:00', 'searchPluseState': False, 'name': 'SMEG_Блендер стац_28.10.2024', 'unitedParams': [{'catalogCPM': 0, 'searchCPM': 150, 'subject': {'id': 614, 'name': 'блендеры'}, 'MenuIds': None, 'menus': [{'id': 0, 'name': ''}], 'nms': [202777102, 149739854, 153936020, 208929076, 174090162]}], 'dailyBudget': 0, 'advertId': 20955474, 'status': 7, 'type': 9, 'paymentType': ''}]


In [29]:
# Flatten the 'unitedParams' field
df_param = pd.json_normalize(
    data,
    record_path='unitedParams',  # Path to the nested list
    meta=[
        'endTime', 'createTime', 'changeTime', 'startTime', 'searchPluseState',
        'name', 'dailyBudget', 'advertId', 'status', 'type', 'paymentType'
    ],  # Include these fields from the parent level
    meta_prefix='parent_',  # Add a prefix to meta fields to avoid conflicts
    sep='_'
)
df_param

,catalogCPM,searchCPM,MenuIds,menus,nms,subject_id,subject_name,parent_endTime,parent_createTime,parent_changeTime,parent_startTime,parent_searchPluseState,parent_name,parent_dailyBudget,parent_advertId,parent_status,parent_type,parent_paymentType
0,0,150,None,"[{'id': 0, 'name': ''}]","[202777102, 149739854, 153936020, 208929076, 1...",614,блендеры,2024-10-31T19:09:55.666795+03:00,2024-10-28T18:49:38.583434+03:00,2024-10-31T19:09:55.666795+03:00,2024-10-31T12:15:15.669999+03:00,False,SMEG_Блендер стац_28.10.2024,0,20955474,7,9,


In [33]:
# Explode the 'nms' field to create a row for each value in the list
df_exploded = df_param.explode('nms')
df_exploded

,catalogCPM,searchCPM,MenuIds,menus,nms,subject_id,subject_name,parent_endTime,parent_createTime,parent_changeTime,parent_startTime,parent_searchPluseState,parent_name,parent_dailyBudget,parent_advertId,parent_status,parent_type,parent_paymentType
0,0,150,None,"[{'id': 0, 'name': ''}]",202777102,614,блендеры,2024-10-31T19:09:55.666795+03:00,2024-10-28T18:49:38.583434+03:00,2024-10-31T19:09:55.666795+03:00,2024-10-31T12:15:15.669999+03:00,False,SMEG_Блендер стац_28.10.2024,0,20955474,7,9,
0,0,150,None,"[{'id': 0, 'name': ''}]",149739854,614,блендеры,2024-10-31T19:09:55.666795+03:00,2024-10-28T18:49:38.583434+03:00,2024-10-31T19:09:55.666795+03:00,2024-10-31T12:15:15.669999+03:00,False,SMEG_Блендер стац_28.10.2024,0,20955474,7,9,
0,0,150,None,"[{'id': 0, 'name': ''}]",153936020,614,блендеры,2024-10-31T19:09:55.666795+03:00,2024-10-28T18:49:38.583434+03:00,2024-10-31T19:09:55.666795+03:00,2024-10-31T12:15:15.669999+03:00,False,SMEG_Блендер стац_28.10.2024,0,20955474,7,9,
0,0,150,None,"[{'id': 0, 'name': ''}]",208929076,614,блендеры,2024-10-31T19:09:55.666795+03:00,2024-10-28T18:49:38.583434+03:00,2024-10-31T19:09:55.666795+03:00,2024-10-31T12:15:15.669999+03:00,False,SMEG_Блендер стац_28.10.2024,0,20955474,7,9,
0,0,150,None,"[{'id': 0, 'name': ''}]",174090162,614,блендеры,2024-10-31T19:09:55.666795+03:00,2024-10-28T18:49:38.583434+03:00,2024-10-31T19:09:55.666795+03:00,2024-10-31T12:15:15.669999+03:00,False,SMEG_Блендер стац_28.10.2024,0,20955474,7,9,


In [45]:
# Define the API endpoint
url = "https://advert-api.wildberries.ru/adv/v1/upd"

# Define the headers (replace 'your_api_token' with your actual API token)
headers = {
    "Authorization": KeyGiper,  # Replace with your API token
    "Content-Type": "application/json"
}

# Define the query parameters
query_params = {
    "from": "2025-02-17",  # Start date (replace with your desired date)
    "to": "2025-02-23"     # End date (replace with your desired date)
}

# Make the GET request
response = requests.get(url, headers=headers, params=query_params)

# Check the response status
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()
    
    # Transform the data into a pandas DataFrame
    df = pd.DataFrame(data)
    
    # Display the DataFrame
    print("DataFrame:")
    print(df)
else:
    print(f"Error: {response.status_code}, {response.text}")

DataFrame:
                              updTime                                campName  \
0           2025-02-23T23:59:59+03:00  Braun_6 серия_Электробритвы_31.01.2025   
1           2025-02-23T23:59:59+03:00             AND_Тонометр мех_27.01.2025   
2           2025-02-23T23:59:59+03:00               CASO_Яйцеварки_27.01.2025   
3           2025-02-23T23:59:59+03:00              ORAL-B_Щетки io_31.05.2025   
4           2025-02-23T23:59:59+03:00      Braun_Электробритвы_30.01.2025_РКП   
..                                ...                                     ...   
244  2025-02-17T14:42:58.636602+03:00      ELIKOR_Вытяжки 50см_29.01.2025_РКП   
245  2025-02-17T14:21:05.370467+03:00              Gillette_Наборы_20.11.2024   
246   2025-02-17T13:53:45.32265+03:00              ORAL-B_Щетки io_31.05.2025   
247  2025-02-17T11:00:35.931673+03:00  Braun_9 серия_Электробритвы_31.01.2025   
248  2025-02-17T10:55:14.005103+03:00     AND_Тонометр на запястье_27.01.2025   

    paymentType 

In [46]:
df

,updTime,campName,paymentType,updNum,updSum,advertId,advertType,advertStatus
0,2025-02-23T23:59:59+03:00,Braun_6 серия_Электробритвы_31.01.2025,Баланс,225639476,184,22950974,9,11
1,2025-02-23T23:59:59+03:00,AND_Тонометр мех_27.01.2025,Баланс,225639476,545,22845877,8,9
2,2025-02-23T23:59:59+03:00,CASO_Яйцеварки_27.01.2025,Баланс,225639476,364,22844946,8,9
3,2025-02-23T23:59:59+03:00,ORAL-B_Щетки io_31.05.2025,Баланс,225639476,1008,22954264,8,9
4,2025-02-23T23:59:59+03:00,Braun_Электробритвы_30.01.2025_РКП,Баланс,225639476,851,22931827,8,9
...,...,...,...,...,...,...,...,...
244,2025-02-17T14:42:58.636602+03:00,ELIKOR_Вытяжки 50см_29.01.2025_РКП,Баланс,224582459,323,22902062,8,9
245,2025-02-17T14:21:05.370467+03:00,Gillette_Наборы_20.11.2024,Баланс,224582459,1500,21526230,8,9
246,2025-02-17T13:53:45.32265+03:00,ORAL-B_Щетки io_31.05.2025,Баланс,224582459,1500,22954264,8,9
247,2025-02-17T11:00:35.931673+03:00,Braun_9 серия_Электробритвы_31.01.2025,Баланс,224582459,203,22951156,9,9


In [48]:
filename = "Promotion.xlsx"
df.to_excel(filename, index=False)

In [49]:
unique_change_times = df['paymentType'].unique()
print(unique_change_times)

['Баланс' 'Кэшбэк']


In [107]:
# API endpoint
url = "https://advert-api.wildberries.ru/adv/v2/fullstats"

# Define the date interval
date_interval = {
    "begin": "2025-01-01",  # Replace with your start date
    "end": "2025-01-31"     # Replace with your end date
}

# Create chunks of 100 campaign IDs each
chunk_size = 100

## Guten

In [108]:
# Create chunks of 100 campaign IDs each
campaign_chunks = [campaigns_guten['advertId'][i:i + chunk_size].tolist() for i in range(0, len(campaigns_guten), chunk_size)]

# List to store all campaign data
all_campaign_data_guten = []

# Iterate over each chunk and send a POST request
for idx, chunk in enumerate(campaign_chunks):
    #print(f"Processing Chunk {idx + 1}: {chunk}")
    
    # Construct the payload for the current chunk
    payload = [{"id": campaign_id, "interval": date_interval} for campaign_id in chunk]
    
    # Send the POST request
    response = requests.post(url, headers=headers_guten, json=payload)
    
    # Add a delay to avoid hitting API rate limits
    time.sleep(65)
    
    # Check the response status
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        all_campaign_data_guten.extend(data)
        #print(f"Response for Chunk {idx + 1}: {json.dumps(data, indent=4)}")
        print("Data retrived successfully")
    else:
        print(f"Error for Chunk {idx + 1}: {response.status_code}, {response.text}")

# Combine all campaign data into a DataFrame
campaign_guten_df = pd.json_normalize(all_campaign_data_guten)

# Save the combined data to a text file
with open("response_data_guten.txt", "w", encoding="utf-8") as file:
    json.dump(all_campaign_data_guten, file, indent=4, ensure_ascii=False)

# Display the combined DataFrame
print("Combined Campaign Data")

Data retrived successfully
Combined Campaign Data


## Giper

In [109]:
# Create chunks of 100 campaign IDs each
campaign_chunks = [campaigns_giper['advertId'][i:i + chunk_size].tolist() for i in range(0, len(campaigns_giper), chunk_size)]

# List to store all campaign data
all_campaign_data_giper = []

# Iterate over each chunk and send a POST request
for idx, chunk in enumerate(campaign_chunks):
    #print(f"Processing Chunk {idx + 1}: {chunk}")
    
    # Construct the payload for the current chunk
    payload = [{"id": campaign_id, "interval": date_interval} for campaign_id in chunk]
    
    # Send the POST request
    response = requests.post(url, headers=headers_giper, json=payload)
    
    # Add a delay to avoid hitting API rate limits
    time.sleep(65)
    
    # Check the response status
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        all_campaign_data_giper.extend(data)
        #print(f"Response for Chunk {idx + 1}: {json.dumps(data, indent=4)}")
        print("Data retrived successfully")
    else:
        print(f"Error for Chunk {idx + 1}: {response.status_code}, {response.text}")

# Combine all campaign data into a DataFrame
campaign_giper_df = pd.json_normalize(all_campaign_data_giper)

# Save the combined data to a text file
with open("response_data_guten.txt", "w", encoding="utf-8") as file:
    json.dump(all_campaign_data_giper, file, indent=4, ensure_ascii=False)

# Display the combined DataFrame
print("Combined Campaign Data")

Data retrived successfully
Combined Campaign Data


## Kitchen

In [110]:
# Create chunks of 100 campaign IDs each
campaign_chunks = [campaigns_kitchen['advertId'][i:i + chunk_size].tolist() for i in range(0, len(campaigns_kitchen), chunk_size)]

# List to store all campaign data
all_campaign_data_kitchen = []

# Iterate over each chunk and send a POST request
for idx, chunk in enumerate(campaign_chunks):
    #print(f"Processing Chunk {idx + 1}: {chunk}")
    
    # Construct the payload for the current chunk
    payload = [{"id": campaign_id, "interval": date_interval} for campaign_id in chunk]
    
    # Send the POST request
    response = requests.post(url, headers=headers_kitchen, json=payload)
    
    # Add a delay to avoid hitting API rate limits
    time.sleep(65)
    
    # Check the response status
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        all_campaign_data_kitchen.extend(data)
        #print(f"Response for Chunk {idx + 1}: {json.dumps(data, indent=4)}")
        print("Data retrived successfully")
    else:
        print(f"Error for Chunk {idx + 1}: {response.status_code}, {response.text}")

# Combine all campaign data into a DataFrame
campaign_kitchen_df = pd.json_normalize(all_campaign_data_kitchen)

# Save the combined data to a text file
with open("response_data_guten.txt", "w", encoding="utf-8") as file:
    json.dump(all_campaign_data_kitchen, file, indent=4, ensure_ascii=False)

# Display the combined DataFrame
print("Combined Campaign Data")

Data retrived successfully
Combined Campaign Data


In [111]:
campaign_guten_df

,views,clicks,ctr,cpc,sum,atbs,orders,cr,shks,sum_price,days,boosterStats,advertId,interval.begin,interval.end
0,58029,2211,3.81,9.31,20587.06,226,60,2.71,60,307916,"[{'date': '2025-01-01T03:00:00+03:00', 'views'...","[{'date': '2025-01-01T00:00:00Z', 'nm': 266534...",21821064,2025-01-01,2025-01-31
1,52692,1320,2.51,9.18,12119.16,80,9,0.68,9,72555,"[{'date': '2025-01-09T03:00:00+03:00', 'views'...",NaN,21204193,2025-01-01,2025-01-31
2,110598,8970,8.11,2.01,18038.12,700,74,0.82,75,403828,"[{'date': '2025-01-03T03:00:00+03:00', 'views'...",NaN,19276556,2025-01-01,2025-01-31
3,130490,11623,8.91,2.51,29141.56,838,77,0.66,77,407387,"[{'date': '2025-01-03T03:00:00+03:00', 'views'...",NaN,19276694,2025-01-01,2025-01-31
4,91404,3697,4.04,1.88,6958.14,334,36,0.97,36,274809,"[{'date': '2025-01-01T03:00:00+03:00', 'views'...","[{'date': '2025-01-01T00:00:00Z', 'nm': 241706...",21203996,2025-01-01,2025-01-31
5,54124,1995,3.69,3.25,6487.88,96,18,0.90,18,41185,"[{'date': '2025-01-01T03:00:00+03:00', 'views'...","[{'date': '2025-01-09T00:00:00Z', 'nm': 181893...",21498270,2025-01-01,2025-01-31
6,18084,698,3.86,3.70,2582.59,66,14,2.01,15,66528,"[{'date': '2025-01-01T03:00:00+03:00', 'views'...","[{'date': '2025-01-14T00:00:00Z', 'nm': 462925...",18870619,2025-01-01,2025-01-31
7,40674,1831,4.50,3.78,6923.54,195,34,1.86,34,94678,"[{'date': '2025-01-27T03:00:00+03:00', 'views'...","[{'date': '2025-01-27T00:00:00Z', 'nm': 159121...",22846516,2025-01-27,2025-01-31
8,28240,1005,3.56,4.21,4234.44,62,6,0.60,6,118442,"[{'date': '2025-01-01T03:00:00+03:00', 'views'...",NaN,21203075,2025-01-01,2025-01-31
9,73807,2548,3.45,5.85,14902.44,155,18,0.71,18,172058,"[{'date': '2025-01-01T03:00:00+03:00', 'views'...",NaN,21203007,2025-01-01,2025-01-31


In [210]:
# API endpoint
url = "https://advert-api.wildberries.ru/adv/v2/fullstats"

# Request headers
headers = {
    "Authorization": KeyGuten,
    "Content-Type": "application/json"
}

# Define the payload using the correct schema
payload = [
    {
        "id": 21554118,  # Replace with your campaign ID
           "interval": {
                "begin": "2025-02-17",
                "end": "2025-02-23"
            } # Array of dates
    }
]

# Send POST request
response = requests.post(url, headers=headers, json=payload)

# Check if the request was successful
if response.status_code == 200:
    print("Request successful!")
    data = response.json()
    
        # Save the response data into a text file
    with open("response_data.txt", "w", encoding="utf-8") as file:
        json.dump(data, file, indent=4, ensure_ascii=False)  # Pretty-print JSON into the file
    
    df = pd.DataFrame(data)
    print(json.dumps(data, indent=4))  # Pretty-print the JSON response
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")
    print(f"Response: {response.text}")

Request successful!
[
    {
        "interval": {
            "begin": "2025-02-17",
            "end": "2025-02-23"
        },
        "views": 34078,
        "clicks": 1187,
        "ctr": 3.48,
        "cpc": 12.88,
        "sum": 15287.5,
        "atbs": 136,
        "orders": 27,
        "cr": 2.27,
        "shks": 27,
        "sum_price": 78812,
        "days": [
            {
                "date": "2025-02-17T03:00:00+03:00",
                "views": 4649,
                "clicks": 202,
                "ctr": 4.35,
                "cpc": 2.53,
                "sum": 510.72,
                "atbs": 29,
                "orders": 6,
                "cr": 2.97,
                "shks": 6,
                "sum_price": 17382,
                "apps": [
                    {
                        "views": 400,
                        "clicks": 7,
                        "ctr": 1.75,
                        "cpc": 6.12,
                        "sum": 42.83,
                        "at

In [211]:
df

,interval,views,clicks,ctr,cpc,sum,atbs,orders,cr,shks,sum_price,days,boosterStats,advertId
0,"{'begin': '2025-02-17', 'end': '2025-02-23'}",34078,1187,3.48,12.88,15287.5,136,27,2.27,27,78812,"[{'date': '2025-02-17T03:00:00+03:00', 'views'...","[{'date': '2025-02-17T00:00:00Z', 'nm': 171562...",21554118


In [212]:
# Flatten the JSON data
flattened_data = []

for entry in data:
    advertId = entry["advertId"]  # Extract the advertId
    for day in entry["days"]:
        date = day["date"]
        for app in day["apps"]:
            for nm in app["nm"]:
                flattened_data.append({
                    "date": date,
                    "nmId": nm["nmId"],
                    "name": nm["name"],
                    "views": nm["views"],
                    "clicks": nm["clicks"],
                    "ctr": nm["ctr"],
                    "cpc": nm["cpc"],
                    "sum": nm["sum"],
                    "atbs": nm["atbs"],
                    "orders": nm["orders"],
                    "cr": nm["cr"],
                    "shks": nm["shks"],
                    "sum_price": nm["sum_price"],
                    "advertId": advertId  # Add advertId to each row
                })

# Create a DataFrame
df = pd.DataFrame(flattened_data)
# Convert the 'date' column to datetime
df["date"] = pd.to_datetime(df["date"])
# Remove timezone information (make it timezone-naive)
df["date"] = df["date"].dt.tz_localize(None)
# Display the DataFrame
df


,date,nmId,name,views,clicks,ctr,cpc,sum,atbs,orders,cr,shks,sum_price,advertId
0,2025-02-17 03:00:00,171562375,Ингалятор небулайзер компрессорный детский CN-...,268,6,2.24,4.80,28.78,0,0,0.00,0,0,21554118
1,2025-02-17 03:00:00,171562370,Ингалятор небулайзер компрессорный детский CN-...,132,1,0.76,14.05,14.05,0,0,0.00,0,0,21554118
2,2025-02-17 03:00:00,171562370,Ингалятор небулайзер компрессорный детский CN-...,867,40,4.61,2.36,94.49,6,1,2.50,1,2833,21554118
3,2025-02-17 03:00:00,171562375,Ингалятор небулайзер компрессорный детский CN-...,2230,108,4.84,2.20,237.81,15,3,2.78,3,8787,21554118
4,2025-02-17 03:00:00,171562375,Ингалятор небулайзер компрессорный детский CN-...,861,41,4.76,2.42,99.38,6,1,2.44,1,2929,21554118
5,2025-02-17 03:00:00,171562370,Ингалятор небулайзер компрессорный детский CN-...,291,6,2.06,6.04,36.21,2,1,16.67,1,2833,21554118
6,2025-02-18 03:00:00,171562370,Ингалятор небулайзер компрессорный детский CN-...,190,4,2.11,20.08,80.32,1,0,0.00,0,0,21554118
7,2025-02-18 03:00:00,171562375,Ингалятор небулайзер компрессорный детский CN-...,296,6,2.03,19.80,118.79,1,1,16.67,1,2929,21554118
8,2025-02-18 03:00:00,171562375,Ингалятор небулайзер компрессорный детский CN-...,2314,107,4.62,8.76,937.55,14,5,4.67,5,14645,21554118
9,2025-02-18 03:00:00,171562370,Ингалятор небулайзер компрессорный детский CN-...,1320,39,2.95,14.63,570.56,5,1,2.56,1,2833,21554118


In [112]:
# Flatten the JSON data
flattened_data_guten = []
flattened_data_giper = []
flattened_data_kitchen = []


for entry in all_campaign_data_guten:
    advertId = entry["advertId"]  # Extract the advertId
    for day in entry["days"]:
        date = day["date"]
        for app in day["apps"]:
            for nm in app["nm"]:
                flattened_data_guten.append({
                    "date": date,
                    "nmId": nm["nmId"],
                    "name": nm["name"],
                    "views": nm["views"],
                    "clicks": nm["clicks"],
                    "ctr": nm["ctr"],
                    "cpc": nm["cpc"],
                    "sum": nm["sum"],
                    "atbs": nm["atbs"],
                    "orders": nm["orders"],
                    "cr": nm["cr"],
                    "shks": nm["shks"],
                    "sum_price": nm["sum_price"],
                    "advertId": advertId  # Add advertId to each row
                })
                
for entry in all_campaign_data_giper:
    advertId = entry["advertId"]  # Extract the advertId
    for day in entry["days"]:
        date = day["date"]
        for app in day["apps"]:
            for nm in app["nm"]:
                flattened_data_giper.append({
                    "date": date,
                    "nmId": nm["nmId"],
                    "name": nm["name"],
                    "views": nm["views"],
                    "clicks": nm["clicks"],
                    "ctr": nm["ctr"],
                    "cpc": nm["cpc"],
                    "sum": nm["sum"],
                    "atbs": nm["atbs"],
                    "orders": nm["orders"],
                    "cr": nm["cr"],
                    "shks": nm["shks"],
                    "sum_price": nm["sum_price"],
                    "advertId": advertId  # Add advertId to each row
                })

for entry in all_campaign_data_kitchen:
    advertId = entry["advertId"]  # Extract the advertId
    for day in entry["days"]:
        date = day["date"]
        for app in day["apps"]:
            for nm in app["nm"]:
                flattened_data_kitchen.append({
                    "date": date,
                    "nmId": nm["nmId"],
                    "name": nm["name"],
                    "views": nm["views"],
                    "clicks": nm["clicks"],
                    "ctr": nm["ctr"],
                    "cpc": nm["cpc"],
                    "sum": nm["sum"],
                    "atbs": nm["atbs"],
                    "orders": nm["orders"],
                    "cr": nm["cr"],
                    "shks": nm["shks"],
                    "sum_price": nm["sum_price"],
                    "advertId": advertId  # Add advertId to each row
                })

# Create a DataFrame
df_guten = pd.DataFrame(flattened_data_guten)
df_giper = pd.DataFrame(flattened_data_giper)
df_kitchen = pd.DataFrame(flattened_data_kitchen)
# Convert the 'date' column to datetime
df_guten["date"] = pd.to_datetime(df_guten["date"])
df_giper["date"] = pd.to_datetime(df_giper["date"])
df_kitchen["date"] = pd.to_datetime(df_kitchen["date"])
# Remove timezone information (make it timezone-naive)
df_guten["date"] = df_guten["date"].dt.tz_localize(None)
df_giper["date"] = df_giper["date"].dt.tz_localize(None)
df_kitchen["date"] = df_kitchen["date"].dt.tz_localize(None)

# Display the DataFrame
df_kitchen

,date,nmId,name,views,clicks,ctr,cpc,sum,atbs,orders,cr,shks,sum_price,advertId
0,2025-01-01 03:00:00,268093569,"Планетарный миксер Artisan 5KSM70JPXEER 6.6 л,...",1,0,0.00,0.00,0.05,0,0,0.0,0,0,21008249
1,2025-01-01 03:00:00,38425737,Планетарный миксер кухонный электрический 4.8 ...,1,0,0.00,0.00,0.05,0,0,0.0,0,0,21008249
2,2025-01-01 03:00:00,38568502,Планетарный миксер кухонный электрический 4.8 ...,1,0,0.00,0.00,0.05,0,0,0.0,0,0,21008249
3,2025-01-02 03:00:00,268093569,"Планетарный миксер Artisan 5KSM70JPXEER 6.6 л,...",1,0,0.00,0.00,0.05,0,0,0.0,0,0,21008249
4,2025-01-02 03:00:00,38331914,"Планетарный миксер ARTISAN 4.8 л, кремовый, 5K...",1,0,0.00,0.00,0.05,0,0,0.0,0,0,21008249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1412,2025-01-31 03:00:00,226627021,Тостер для хлеба с поддоном для крошек 5KMT210...,17,0,0.00,0.00,2.43,0,0,0.0,0,0,21006669
1413,2025-01-31 03:00:00,226627021,Тостер для хлеба с поддоном для крошек 5KMT210...,21,0,0.00,0.00,3.29,0,0,0.0,0,0,21006669
1414,2025-01-31 03:00:00,245313750,Тостер на 4 ломтика 5KMT4109E,67,0,0.00,0.00,10.85,0,0,0.0,0,0,21006669
1415,2025-01-31 03:00:00,54807104,"Тостер ARTISAN, чернильный синий, 5KMT2204",89,3,3.37,4.38,13.14,0,0,0.0,0,0,21006669


In [364]:
unique_change_times = df['advertId'].nunique()
print(unique_change_times)

61


In [113]:
# Group by 'date' (day only) and 'nmId', summing numeric columns except 'advertId'
df_grouped_guten = (
    df_guten.groupby([df_guten["date"].dt.date, "nmId"], as_index=False)
    .agg({
        "date": "first",  # Keep the first date (to retain the day)
        "name": "first",  # Keep the first name (or customize this logic)
        "views": "sum",
        "clicks": "sum",
        "ctr": "mean",  # Sum or average, depending on your needs
        "cpc": "mean",  # Use mean for 'cpc' (cost per click)
        "sum": "sum",
        "atbs": "sum",
        "orders": "sum",
        "cr": "mean",  # Use mean for 'cr' (conversion rate)
        "shks": "sum",
        "sum_price": "sum",
        "advertId": "first"  # Keep the first 'advertId' (no summing)
    })
)

# Group by 'date' (day only) and 'nmId', summing numeric columns except 'advertId'
df_grouped_giper = (
    df_giper.groupby([df_giper["date"].dt.date, "nmId"], as_index=False)
    .agg({
        "date": "first",  # Keep the first date (to retain the day)
        "name": "first",  # Keep the first name (or customize this logic)
        "views": "sum",
        "clicks": "sum",
        "ctr": "mean",  # Sum or average, depending on your needs
        "cpc": "mean",  # Use mean for 'cpc' (cost per click)
        "sum": "sum",
        "atbs": "sum",
        "orders": "sum",
        "cr": "mean",  # Use mean for 'cr' (conversion rate)
        "shks": "sum",
        "sum_price": "sum",
        "advertId": "first"  # Keep the first 'advertId' (no summing)
    })
)

# Group by 'date' (day only) and 'nmId', summing numeric columns except 'advertId'
df_grouped_kitchen = (
    df_kitchen.groupby([df_kitchen["date"].dt.date, "nmId"], as_index=False)
    .agg({
        "date": "first",  # Keep the first date (to retain the day)
        "name": "first",  # Keep the first name (or customize this logic)
        "views": "sum",
        "clicks": "sum",
        "ctr": "mean",  # Sum or average, depending on your needs
        "cpc": "mean",  # Use mean for 'cpc' (cost per click)
        "sum": "sum",
        "atbs": "sum",
        "orders": "sum",
        "cr": "mean",  # Use mean for 'cr' (conversion rate)
        "shks": "sum",
        "sum_price": "sum",
        "advertId": "first"  # Keep the first 'advertId' (no summing)
    })
)

# Rename the 'date' column to 'day' for clarity
df_grouped_guten.rename(columns={"date": "day"}, inplace=True)
df_grouped_giper.rename(columns={"date": "day"}, inplace=True)
df_grouped_kitchen.rename(columns={"date": "day"}, inplace=True)

df_grouped_guten['Project'] = 'WB-GutenTech'
df_grouped_giper['Project'] = 'WB-ГиперМаркет'
df_grouped_kitchen['Project'] = 'WB-KitchenAid'

# Concatenate the DataFrames
df_grouped_combined_campaigns = pd.concat([df_grouped_guten, df_grouped_giper, df_grouped_kitchen], ignore_index=True)
df_grouped_combined_campaigns['Marketplace'] = 'Wildberries'

# Display the grouped DataFrame
df_grouped_combined_campaigns

,nmId,day,name,views,clicks,ctr,cpc,sum,atbs,orders,cr,shks,sum_price,advertId,Project,Marketplace
0,25623718,2025-01-01 03:00:00,Бритва мужская электрическая S3 300BT с триммером,99,3,3.253333,1.230000,7.64,0,0,0.0,0,0,21202453,WB-GutenTech,Wildberries
1,25624340,2025-01-01 03:00:00,Электробритва Series 3 300s,397,12,2.416667,1.276667,27.52,0,0,0.0,0,0,21202453,WB-GutenTech,Wildberries
2,25628178,2025-01-01 03:00:00,Бритва мужская электрическая S3 300s,40,0,0.000000,0.000000,3.13,0,0,0.0,0,0,21202453,WB-GutenTech,Wildberries
3,25725047,2025-01-01 03:00:00,Триммер для бороды и усов BT3222,46,1,2.776667,0.316667,3.90,0,0,0.0,0,0,21212396,WB-GutenTech,Wildberries
4,25994579,2025-01-01 03:00:00,Эпилятор женский электрический Silk-epil 1-170,1,0,0.000000,0.000000,0.04,0,0,0.0,0,0,21203996,WB-GutenTech,Wildberries
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5600,243235996,2025-01-31 03:00:00,"Чайник электрический 1,7 л с регулировкой темп...",2,0,0.000000,0.000000,0.18,0,0,0.0,0,0,21008536,WB-KitchenAid,Wildberries
5601,243235997,2025-01-31 03:00:00,"Чайник электрический 1,7 л с регулировкой темп...",67,2,2.316667,2.700000,8.95,0,0,0.0,0,0,21008536,WB-KitchenAid,Wildberries
5602,243235999,2025-01-31 03:00:00,"Чайник электрический 1,7 л с регулировкой темп...",153,7,5.013333,2.240000,16.74,0,0,0.0,0,0,21008536,WB-KitchenAid,Wildberries
5603,245313750,2025-01-31 03:00:00,Тостер на 4 ломтика 5KMT4109E,112,0,0.000000,0.000000,18.89,0,0,0.0,0,0,21006669,WB-KitchenAid,Wildberries


In [114]:
# Merge the grouped DataFrame with the filtered_df to add additional columns
df_final = df_grouped_combined_campaigns.merge(
    filtered_df[["advertId", "endTime", "createTime", "startTime","name", "status", "type"]],
    on="advertId",
    how="left"
)

# Drop the columns 'ctr', 'cpc', and 'cr'
df_final = df_final.drop(columns=["ctr", "cpc", "cr"])

# Rename the columns 'name_x' and 'name_y'
df_final.rename(
    columns={
        "name_x": "name_product",  # Rename 'name_x' to 'name_product'
        "name_y": "name_campaign"  # Rename 'name_y' to 'name_campaign'
    },
    inplace=True
)

# Display the final DataFrame
df_final

,nmId,day,name_product,views,clicks,sum,atbs,orders,shks,sum_price,advertId,Project,Marketplace,endTime,createTime,startTime,name_campaign,status,type
0,25623718,2025-01-01 03:00:00,Бритва мужская электрическая S3 300BT с триммером,99,3,7.64,0,0,0,0,21202453,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания
1,25624340,2025-01-01 03:00:00,Электробритва Series 3 300s,397,12,27.52,0,0,0,0,21202453,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания
2,25628178,2025-01-01 03:00:00,Бритва мужская электрическая S3 300s,40,0,3.13,0,0,0,0,21202453,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания
3,25725047,2025-01-01 03:00:00,Триммер для бороды и усов BT3222,46,1,3.90,0,0,0,0,21212396,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Триммеры_07.11.2024,Идут показы,Автоматическая кампания
4,25994579,2025-01-01 03:00:00,Эпилятор женский электрический Silk-epil 1-170,1,0,0.04,0,0,0,0,21203996,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Эпиляторы_07.11.2024_РКП,Идут показы,Автоматическая кампания
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5600,243235996,2025-01-31 03:00:00,"Чайник электрический 1,7 л с регулировкой темп...",2,0,0.18,0,0,0,0,21008536,WB-KitchenAid,Wildberries,2100-01-01,2024-10-30,2025-01-09,Чайники_30.10.2024,Идут показы,Автоматическая кампания
5601,243235997,2025-01-31 03:00:00,"Чайник электрический 1,7 л с регулировкой темп...",67,2,8.95,0,0,0,0,21008536,WB-KitchenAid,Wildberries,2100-01-01,2024-10-30,2025-01-09,Чайники_30.10.2024,Идут показы,Автоматическая кампания
5602,243235999,2025-01-31 03:00:00,"Чайник электрический 1,7 л с регулировкой темп...",153,7,16.74,0,0,0,0,21008536,WB-KitchenAid,Wildberries,2100-01-01,2024-10-30,2025-01-09,Чайники_30.10.2024,Идут показы,Автоматическая кампания
5603,245313750,2025-01-31 03:00:00,Тостер на 4 ломтика 5KMT4109E,112,0,18.89,0,0,0,0,21006669,WB-KitchenAid,Wildberries,2100-01-01,2024-10-30,2025-01-09,Тостеры_30.10.2024,Идут показы,Автоматическая кампания


In [ ]:
unique_change_times = df_grouped_combined_campaigns['advertId'].nunique()
print(unique_change_times)

55


In [ ]:
filename = "Compaings.xlsx"
df_final.to_excel(filename, index=False)

In [68]:
# Initialize an empty list to store flattened data
flattened_data = []

# Loop through the nested structure to extract relevant data
for day in data[0]['days']:
    for app in day['apps']:
        app_type = app['appType']
        for nm in app['nm']:
            flattened_data.append({
                'nmId': nm['nmId'],
                'appType': app_type
            })

# Convert the flattened data into a DataFrame
df = pd.DataFrame(flattened_data)

# Group by nmId and calculate the sum of appType
result = df.groupby('nmId', as_index=False).agg({'appType': 'sum'})

# Display the resulting DataFrame
print(result)

        nmId  appType
0  135577637      679
1  137609254      679
2  137609323      679
3  137609342      679
4  137609383      485
5  137609392      679
6  137609742      291


In [ ]:
# Create chunks of 50 campaign IDs each
chunk_size = 50
campaign_chunks = [df['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df), chunk_size)]

# Define the API endpoint
url = "https://advert-api.wildberries.ru/adv/v1/promotion/adverts"

# Define the query parameters
query_params = {
    "order": "change",  # Order by the "change" field
}

# Define the headers (replace 'your_api_token' with your actual API token)
headers = {
    "Authorization": KeyGuten,  # Replace with your API token
    "Content-Type": "application/json"
}

# Iterate over each chunk and send a POST request
for idx, chunk in enumerate(campaign_chunks):
    print(f"Processing Chunk {idx + 1}: {chunk}")
    
    # Send the POST request
    response = requests.post(url, params=query_params, json=chunk, headers=headers)
    time.sleep(2)
    # Check the response status
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        print(f"Response for Chunk {idx + 1}: {data}")
    else:
        print(f"Error for Chunk {idx + 1}: {response.status_code}, {response.text}")

[[20397145, 20397161, 20954456, 20955117, 21041143, 21041194, 21041248, 21041312, 21041387, 21577733, 21577749, 21578389, 21624850, 21698516, 22192773, 23231292, 7202847, 8004530, 8743562, 9496209, 9496385, 9591336, 11296204, 11296333, 16255090, 16301721, 16301819, 16912359, 16912972, 16913380, 16913696, 16913774, 16914258, 16917733, 16917791, 16918232, 16918415, 16919369, 16920802, 16923161, 16923906, 16923933, 16924292, 17266442, 17563748, 17563873, 17563938, 17687135, 17687241, 17687293], [17687357, 17688586, 17688635, 18640342, 18870731, 18870828, 19262099, 19263710, 19263824, 19265149, 19265205, 19626785, 19662812, 19904919, 19994021, 20039654, 20139599, 20175678, 20334946, 20354793, 20397195, 21040251, 21040485, 21040635, 21040756, 21040856, 21040946, 21385613, 21386362, 21482495, 21482904, 21498202, 8618594, 8743328, 462902, 462934, 462959, 462991, 463002, 463125, 463138, 463196, 463211, 463364, 480144, 480165, 480178, 480275, 480291, 480347], [480356, 480455, 480513, 480527, 48

' # Define the API endpoint\nurl = "https://advert-api.wildberries.ru/adv/v1/promotion/adverts"\n\n# Define the query parameters\nquery_params = {\n    "status": 9,  # Example: campaigns with status "9 - идут показы"\n    "type": 8,    # Example: automatic campaigns\n    "order": "change",  # Order by the "change" field\n    "direction": "asc"  # Ascending order\n}\n\n# Define the headers (replace \'your_api_token\' with your actual API token)\nheaders = {\n    "Authorization": "Bearer your_api_token",  # Replace with your API token\n    "Content-Type": "application/json"\n}\n\n# Iterate over each chunk and send a POST request\nfor idx, chunk in enumerate(campaign_chunks):\n    print(f"Processing Chunk {idx + 1}: {chunk}")\n    \n    # Send the POST request\n    response = requests.post(url, params=query_params, json=chunk, headers=headers)\n    \n    # Check the response status\n    if response.status_code == 200:\n        # Parse the JSON response\n        data = response.json()\n   

In [115]:
password = os.getenv('ClickHouse')

In [116]:
# Define connection parameters
client = get_client(
    host='rc1a-vk5i3icccvmfk6cm.mdb.yandexcloud.net',  # Your Yandex Cloud ClickHouse host
    port=8443,                                          # Yandex Cloud uses port 8443 for HTTPS
    username='user1',                           # Your ClickHouse username
    password= password,                           # Your ClickHouse password
    database='user1',                            # Your database name
    secure=True,                                        # Use HTTPS
    verify=False                                        # Disable SSL certificate verification 
    # Define the data to insert
)

In [106]:
# Define the table name
table_name = 'campaign_data_wb'

# Convert DataFrame to a list of tuples for bulk insertion
data = [tuple(row) for row in df_final.to_numpy()]

# Define the column names
column_names = [
    'nmId', 'day', 'name_product', 'views', 'clicks', 'sum', 'atbs', 'orders', 'shks', 'sum_price',
    'advertId', 'Project', 'Marketplace', 'endTime', 'createTime', "startTime", 'name_campaign', 'status', 'type'
]

# Use the insert method for bulk insertion
client.insert(table_name, data, column_names=column_names)

print("Data inserted successfully!")

Data inserted successfully!
